In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
from datetime import timedelta

In [ ]:
df = pd.read_csv("Retail.csv")
df.head()

/tmp/ipython-input-396735732.py:1: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Retail.csv")


,Invoice,StockCode,Description,Quantity,InvoiceDate,Year,Month,InvoiceTime,Price,TotalAmount,Customer ID,Country
0,489437,21360,JOY LARGE WOOD LETTERS,1,2009-12-01,2009,Dec,09:08,9.95,9.95,15362,United Kingdom
1,489444,POST,POSTAGE,1,2009-12-01,2009,Dec,09:55,141.00,141.00,12636,USA
2,489445,21299,EDWARDIAN TOILET ROLL UNIT,1,2009-12-01,2009,Dec,09:57,12.75,12.75,17519,United Kingdom
3,489447,POST,POSTAGE,1,2009-12-01,2009,Dec,10:10,130.00,130.00,12362,Belgium
4,489488,84031B,CHARLIE LOLA BLUE HOT WATER BOTTLE,1,2009-12-01,2009,Dec,10:59,4.65,4.65,17238,United Kingdom


In [ ]:
print(df.columns)

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Year', 'Month', 'InvoiceTime', 'Price', 'TotalAmount', 'Customer ID',
       'Country'],
      dtype='object')


In [ ]:
df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
print(df.columns)

Index(['invoice', 'stockcode', 'description', 'quantity', 'invoicedate',
       'year', 'month', 'invoicetime', 'price', 'totalamount', 'customer_id',
       'country'],
      dtype='object')


In [ ]:
df['invoicedate'] = pd.to_datetime(df['invoicedate'])

In [ ]:
df['revenue'] = df['quantity'] * df['price']

In [ ]:
rfm = df.groupby('customer_id').agg({
    'invoicedate': 'max',      # Last purchase date
    'stockcode': 'nunique',    # Frequency
    'revenue': 'sum'          # Monetary
}).reset_index()

In [ ]:
rfm.columns = ['customer_id', 'last_purchase_date', 'frequency', 'monetary']

In [ ]:
analysis_date = rfm['last_purchase_date'].max() + pd.Timedelta(days=1)

rfm['recency'] = (analysis_date - rfm['last_purchase_date']).dt.days

In [ ]:
rfm.head()
rfm.describe()

,customer_id,last_purchase_date,frequency,monetary,recency
count,5860.000000,5860,5860.000000,5860.000000,5860.000000
mean,15316.832423,2011-05-17 12:55:46.894198016,105.108874,2956.473053,201.461263
min,12346.000000,2009-12-01 00:00:00,0.000000,2.900000,1.000000
25%,13836.750000,2010-11-23 00:00:00,18.000000,344.492500,26.000000
50%,15315.500000,2011-08-25 00:00:00,48.000000,883.595000,102.000000
75%,16799.250000,2011-11-09 00:00:00,125.000000,2284.647500,377.000000
max,18287.000000,2011-12-04 00:00:00,2982.000000,597336.110000,734.000000
std,1714.531194,NaN,165.150724,14352.382490,207.496838


In [ ]:
rfm['r_score'] = pd.qcut(rfm['recency'], 5, labels=[5,4,3,2,1])
rfm['f_score'] = pd.qcut(rfm['frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['m_score'] = pd.qcut(rfm['monetary'], 5, labels=[1,2,3,4,5])

In [ ]:
rfm['r_score'] = pd.qcut(rfm['recency'], 5, labels=[5,4,3,2,1])
rfm['f_score'] = pd.qcut(rfm['frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
rfm['m_score'] = pd.qcut(rfm['monetary'], 5, labels=[1,2,3,4,5])
rfm['r_score'] = rfm['r_score'].astype(int)
rfm['f_score'] = rfm['f_score'].astype(int)
rfm['m_score'] = rfm['m_score'].astype(int)


In [ ]:

def rfm_segment(row):
    if row['r_score'] >= 4 and row['f_score'] >= 4 and row['m_score'] >= 4:
        return 'Champions'
    elif row['f_score'] >= 4 and row['m_score'] >= 3:
        return 'Loyal Customers'
    elif row['r_score'] >= 4:
        return 'Potential Loyalists'
    elif row['r_score'] <= 2 and row['f_score'] >= 3:
        return 'At Risk'
    else:
        return 'Hibernating'

rfm['Segment'] = rfm.apply(rfm_segment, axis=1)

print(rfm['Segment'].value_counts())

Segment
Hibernating            2136
Champions              1229
Loyal Customers         971
Potential Loyalists     943
At Risk                 581
Name: count, dtype: int64


In [ ]:
rfm.groupby('Segment')['monetary'].mean().sort_values(ascending=False)

,monetary
Segment,
Champions,8864.412576
Loyal Customers,2569.762201
Potential Loyalists,1822.150754
At Risk,839.879587
Hibernating,809.490512


In [ ]:
rfm['clv'] = rfm['monetary'] * rfm['frequency']

In [ ]:
rfm.to_csv("rfm_output.csv", index=False)

In [ ]:
rfm['Segment'].value_counts()

,count
Segment,
Hibernating,2136
Champions,1229
Loyal Customers,971
Potential Loyalists,943
At Risk,581
